# 03 – Pohrana u SQLite bazu (pzap.db)

U ovom koraku spremamo integrirane podatke u SQLite bazu radi lakšeg upita i korištenja u REST API-ju.

Tablice:
- **dt_yearly** (DT: godišnje + daily_minutes)
- **wb_internet_year** (World Bank: internet_pct po godini)
- **integrated_individual** (SMP + agregati DT/WB + TW agregati)


### Definiranje putanja i provjera da CSV postoji

In [ ]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parent 
DATA_PROCESSED = PROJECT_ROOT / "data_processed"
DATA_RAW = PROJECT_ROOT / "data_raw"    

TW_CSV = DATA_RAW / "Time-Wasters on Social Media.csv"
assert TW_CSV.exists(), f"Ne postoji: {TW_CSV}"

tw_raw = pd.read_csv(TW_CSV)

print("TW loaded:", tw_raw.shape)
display(tw_raw.head())

INTEGRATED_CSV = DATA_PROCESSED / "integrated_individual_level.csv"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("INTEGRATED_CSV exists:", INTEGRATED_CSV.exists())

# Učitavanje integriranog CSV-a u DataFrame
integrated_individual = pd.read_csv(INTEGRATED_CSV)
display(integrated_individual.head())
print("integrated_individual shape:", integrated_individual.shape)


TW_CSV exists: True


,UserID,Age,Gender,Location,Income,Debt,Owns Property,Profession,Demographics,Platform,...,ProductivityLoss,Satisfaction,Watch Reason,DeviceType,OS,Watch Time,Self Control,Addiction Level,CurrentActivity,ConnectionType
0,1,56,Male,Pakistan,82812,True,True,Engineer,Rural,Instagram,...,3,7,Procrastination,Smartphone,Android,9:00 PM,5,5,Commuting,Mobile Data
1,2,46,Female,Mexico,27999,False,True,Artist,Urban,Instagram,...,5,5,Habit,Computer,Android,5:00 PM,7,3,At school,Wi-Fi
2,3,32,Female,United States,42436,False,True,Engineer,Rural,Facebook,...,6,4,Entertainment,Tablet,Android,2:00 PM,8,2,At home,Mobile Data
3,4,60,Male,Barzil,62963,True,False,Waiting staff,Rural,YouTube,...,3,7,Habit,Smartphone,Android,9:00 PM,5,5,Commuting,Mobile Data
4,5,25,Male,Pakistan,22096,False,True,Manager,Urban,TikTok,...,8,2,Boredom,Smartphone,iOS,8:00 AM,10,0,At home,Mobile Data


tw_raw shape: (1000, 31)
PROJECT_ROOT: d:\Preuzimanja\PZAP_PROJEKT\PAP_PROJEKT
INTEGRATED_CSV exists: True


,age,gender,job_type,daily_social_media_time,number_of_notifications,work_hours_per_day,perceived_productivity_score,actual_productivity_score,stress_level,social_platform_preference,dt_daily_minutes_avg,internet_pct_avg_dt_years,tw_total_time_spent_mean,tw_sessions_mean,tw_engagement_mean
0,56,Male,Unemployed,4.180940,61,6.753558,8.040464,7.291555,4.0,Facebook,129.230769,56.200389,151.406,10.013,4997.159
1,46,Male,Health,3.249603,59,9.169296,5.063368,5.165093,7.0,Twitter,129.230769,56.200389,151.406,10.013,4997.159
2,56,Female,Unemployed,4.381070,60,3.902309,6.420989,5.976408,7.0,TikTok,129.230769,56.200389,151.406,10.013,4997.159
3,36,Female,Education,4.089168,49,6.560467,2.681830,2.446927,4.0,Twitter,129.230769,56.200389,151.406,10.013,4997.159
4,40,Female,Education,4.097401,57,5.839590,3.219022,3.004240,4.0,Instagram,129.230769,56.200389,151.406,10.013,4997.159


integrated_individual shape: (23730, 15)


### Učitavanje dt_yearly i wb_internet_year

In [2]:
import requests

# DT: učitaj + HH:MM -> minutes
dt = pd.read_csv(DATA_RAW / "DT.csv")

def hhmm_to_minutes(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if ":" not in s:
        return None
    hh, mm = s.split(":", 1)
    try:
        return int(hh) * 60 + int(mm)
    except:
        return None

dt_clean = dt.copy()
dt_clean["daily_minutes"] = dt_clean["Daily Time (Hours:Minutes)"].apply(hhmm_to_minutes)
dt_clean = dt_clean.drop(columns=["Daily Time (Hours:Minutes)"])

# Dohvat World Bank indikatora + agregacija po godini
def fetch_worldbank_indicator(indicator: str, pages_limit: int = 3, per_page: int = 20000):
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator}"
    params = {"format": "json", "per_page": per_page, "page": 1}

    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    meta, _ = r.json()

    pages = min(meta.get("pages", 1), pages_limit)

    rows = []
    for page in range(1, pages + 1):
        params["page"] = page
        rp = requests.get(url, params=params, timeout=60)
        rp.raise_for_status()
        _, data_p = rp.json()

        for item in data_p:
            if item is None:
                continue
            rows.append({
                "Year": int(item.get("date")) if item.get("date") else None,
                "internet_pct": item.get("value"),
            })

    return pd.DataFrame(rows)

wb = fetch_worldbank_indicator("IT.NET.USER.ZS", pages_limit=3)

wb_internet_year = (
    wb.dropna(subset=["Year"])
      .groupby("Year", as_index=False)["internet_pct"]
      .mean()
)
# Spajanje DT + WB po godini → dt_yearly
dt_yearly = dt_clean.merge(wb_internet_year, on="Year", how="left")

display(dt_yearly.head())
display(wb_internet_year.head())
print("dt_yearly:", dt_yearly.shape, "wb_internet_year:", wb_internet_year.shape)


,Year,daily_minutes,internet_pct
0,2012,90,38.020353
1,2013,95,40.499988
2,2014,104,43.315020
3,2015,111,46.123280
4,2016,128,49.483999


,Year,internet_pct
0,1960,NaN
1,1961,NaN
2,1962,NaN
3,1963,NaN
4,1964,NaN


dt_yearly: (13, 3) wb_internet_year: (65, 2)


### Kreiranje SQLite i tablica

Radimo: 
- Spajanje na bazu
- Spremanje DataFrame-ova kao tablica u SQLite
- Commit + zatvaranje

In [3]:
import sqlite3

# Spremanje DataFrame-ova u SQLite bazu podataka
DB_PATH = DATA_PROCESSED / "pap.db"
conn = sqlite3.connect(DB_PATH)

# Spremanje tablica
dt_yearly.to_sql("dt_yearly", conn, if_exists="replace", index=False)
wb_internet_year.to_sql("wb_internet_year", conn, if_exists="replace", index=False)
integrated_individual.to_sql("integrated_individual", conn, if_exists="replace", index=False)

conn.commit()
conn.close()

print("SQLite saved:", DB_PATH, "| exists:", DB_PATH.exists())


SQLite saved: d:\Preuzimanja\PZAP_PROJEKT\PAP_PROJEKT\data_processed\pap.db | exists: True


Provjera (SELECT)

In [4]:
import sqlite3

# Pregled podataka iz SQLite baze podataka
conn = sqlite3.connect(DB_PATH)

print(pd.read_sql_query("SELECT COUNT(*) AS n FROM integrated_individual;", conn))
display(pd.read_sql_query("SELECT * FROM dt_yearly ORDER BY Year LIMIT 5;", conn))
display(pd.read_sql_query("SELECT * FROM integrated_individual LIMIT 5;", conn))

conn.close()


       n
0  23730


,Year,daily_minutes,internet_pct
0,2012,90,38.020353
1,2013,95,40.499988
2,2014,104,43.315020
3,2015,111,46.123280
4,2016,128,49.483999


,age,gender,job_type,daily_social_media_time,number_of_notifications,work_hours_per_day,perceived_productivity_score,actual_productivity_score,stress_level,social_platform_preference,dt_daily_minutes_avg,internet_pct_avg_dt_years,tw_total_time_spent_mean,tw_sessions_mean,tw_engagement_mean
0,56,Male,Unemployed,4.180940,61,6.753558,8.040464,7.291555,4.0,Facebook,129.230769,56.200389,151.406,10.013,4997.159
1,46,Male,Health,3.249603,59,9.169296,5.063368,5.165093,7.0,Twitter,129.230769,56.200389,151.406,10.013,4997.159
2,56,Female,Unemployed,4.381070,60,3.902309,6.420989,5.976408,7.0,TikTok,129.230769,56.200389,151.406,10.013,4997.159
3,36,Female,Education,4.089168,49,6.560467,2.681830,2.446927,4.0,Twitter,129.230769,56.200389,151.406,10.013,4997.159
4,40,Female,Education,4.097401,57,5.839590,3.219022,3.004240,4.0,Instagram,129.230769,56.200389,151.406,10.013,4997.159
